In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
import torch


c:\Users\artur\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Preparar el Modelo

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to(device)

tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))


Embedding(50257, 768)

Configuración de LoRA

In [3]:
peft_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


c:\Users\artur\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\tuners\lora\layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Cargar Dataset

In [8]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

def preprocess_function(examples):
    tokenized = tokenizer(
        examples["text"], truncation=True, padding="max_length", max_length=128
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]
train_dataset = tokenized_dataset["train"].select(range(1000))  # Reducido a 500 ejemplos
eval_dataset = tokenized_dataset["validation"].select(range(300))  # Reducido a 100 ejemplos

Configuración de Entrenamiento

In [9]:
training_args = TrainingArguments(
    output_dir="./gpt2_peft_lora",
    dataloader_drop_last=True,
    evaluation_strategy="epoch",  # Mantén la evaluación por época
    save_strategy="epoch",  # Guarda el modelo por época
    num_train_epochs=50,  # Reduce a 5 si el dataset es grande
    logging_steps=100,  # Registra cada 10 pasos para evitar demasiada información
    per_device_train_batch_size=8,  # Usa 4 si tu GPU tiene suficiente memoria
    per_device_eval_batch_size=8,  # Ajusta para evaluación rápida
    learning_rate=1e-4,  # Mantén la misma tasa de aprendizaje
    lr_scheduler_type="cosine",  # Mantén el programador de aprendizaje coseno
    warmup_steps=100,  # Ajusta a ~10% del total de pasos de entrenamiento
    gradient_accumulation_steps=4,  # Simula un lote mayor acumulando gradientes
    fp16=True,  # Usa precisión mixta si tu GPU lo soporta
    weight_decay=0.01,  # Ajusta ligeramente el decaimiento del peso
    run_name="gpt2_peft_lora",  # Nombre del experimento
    dataloader_num_workers=4,  # Más hilos para cargar datos
    logging_dir="./logs",
)


c:\Users\artur\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Inicializar y entrenar

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

trainer.train()


C:\Users\artur\AppData\Local\Temp\ipykernel_22312\2281871093.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
                                                  
  2%|▏         | 32/1550 [01:27<43:23,  1.71s/it]

{'eval_loss': 7.835604190826416, 'eval_runtime': 19.2939, 'eval_samples_per_second': 15.549, 'eval_steps_per_second': 1.97, 'epoch': 1.0}


                                                   
  4%|▍         | 64/1550 [02:56<41:49,  1.69s/it]

{'eval_loss': 5.80400276184082, 'eval_runtime': 19.1651, 'eval_samples_per_second': 15.653, 'eval_steps_per_second': 1.983, 'epoch': 2.0}


                                                   
  6%|▌         | 96/1550 [04:23<40:08,  1.66s/it]

...

                                                     
100%|██████████| 1550/1550 [1:10:17<00:00,  1.89s/it]

{'eval_loss': 1.358924150466919, 'eval_runtime': 18.7765, 'eval_samples_per_second': 15.977, 'eval_steps_per_second': 2.024, 'epoch': 48.45}


100%|██████████| 1550/1550 [1:10:18<00:00,  2.72s/it]


{'train_runtime': 4218.1811, 'train_samples_per_second': 11.853, 'train_steps_per_second': 0.367, 'train_loss': 1.7114721901186052, 'epoch': 48.45}


TrainOutput(global_step=1550, training_loss=1.7114721901186052, metrics={'train_runtime': 4218.1811, 'train_samples_per_second': 11.853, 'train_steps_per_second': 0.367, 'total_flos': 3170255643869184.0, 'train_loss': 1.7114721901186052, 'epoch': 48.448})

Guardar modelo

In [11]:
model.save_pretrained("./gpt2_peft_lora")
tokenizer.save_pretrained("./gpt2_peft_lora")


('./gpt2_peft_lora\\tokenizer_config.json',
 './gpt2_peft_lora\\special_tokens_map.json',
 './gpt2_peft_lora\\vocab.json',
 './gpt2_peft_lora\\merges.txt',
 './gpt2_peft_lora\\added_tokens.json')

Generar texto

In [12]:
def generate_text(prompt):
    model.eval()  # Asegúrate de que el modelo está en modo evaluación
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=100, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


Pruebas

In [13]:
print(generate_text("The evolution of artificial intelligence has"))
print(generate_text("Once upon a time in a distant galaxy"))
print(generate_text("The future of humanity depends on"))


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The evolution of artificial intelligence has been a topic of debate in the field of artificial intelligence research , with some arguing that artificial intelligence is more intelligent than humans , while others argue that artificial intelligence is more intelligent than humans . Some argue that artificial intelligence is more intelligent than humans , while others argue that artificial intelligence is more intelligent than humans . Some argue that artificial intelligence is more intelligent than humans , while others argue that artificial intelligence is more intelligent than humans . Some argue that artificial intelligence is more intelligent than humans ,


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Once upon a time in a distant galaxy, the stars of the Milky Way and the Milky Way were in a state of flux. The stars of the Milky Way and the Milky Way were in a state of flux. The stars of the Milky Way and the Milky Way were in a state of flux. The stars of the Milky Way and the Milky Way were in a state of flux. The stars of the Milky Way and the Milky Way were in a state of flux. The stars of the Milky Way
The future of humanity depends on the ability of the human species to adapt to the changing environment and to cope with the challenges of life on the planet. The human species must adapt to the changing environment and to cope with the challenges of life on the planet. The human species must adapt to the changing environment and to cope with the challenges of life on the planet.



In [16]:
print(generate_text("the most famous actor in 20th century is"))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


the most famous actor in 20th century is Robert De Niro , who played the lead role in the first film in the series . He was also a member of the cast of the first film in the series . He was also a member of the cast of the first film in the series . He was also a member of the cast of the first film in the series . He was also a member of the cast of the first film in the series . He was also a member of the cast of
